## Flight Difficulty Score Development
## Objective
### Build a daily-level Flight Difficulty Score for ORD departures, ranking flights and classifying them as Difficult, Medium, or Easy.

## Prerequisites
- Python 3.x+, libraries: `pandas`, `numpy`, `statsmodels`
- Install: `pip install -r requirements.txt`
- Input: `Flight Level Data.csv`, `Bag Level Data.csv`, `PNR Flight Level Data.csv`, `PNR Remark Level Data.csv`, `eda_metrics.csv` (optional)

### Approach
- Use OLS regression to predict delay (difficulty proxy), inspired by paper’s ML for delay prediction.
- Features: ground time constraint, bag ratio, load factor, SSR per pax, departure hour, haul, fleet type.
- Rank flights daily by score; classify into tertiles (Difficult: top 33%, Medium: 33-67%, Easy: bottom 33%).

In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime, timedelta

In [18]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('difficulty_score.log'), logging.StreamHandler()]
)
logger = logging.getLogger(__name__)
sns.set_style('whitegrid')
%matplotlib inline
np.random.seed(42)

In [34]:
try:
    flight_df = pd.read_csv('Flight_Level_Data.csv')
    bag_df = pd.read_csv('Bag_Level_Data.csv')
    pnr_df = pd.read_csv('PNR_Flight_Level_Data.csv')
    remark_df = pd.read_csv('PNR_Remark_Level_Data.csv')
    logger.info("Loaded data")
except FileNotFoundError:
    logger.error("Data not found")

flight_df = flight_df[flight_df['scheduled_departure_station_code'] == 'ORD']

2025-10-04 17:13:07,781 - INFO - Loaded data


In [40]:
print("Unique fleet types:", full_df['fleet_type'].unique())
print("\nFleet dummy columns created:", [col for col in full_df.columns if col.startswith('fleet_')])

Unique fleet types: ['ERJ-175' 'B767-300' 'B737-800' 'CRJ-200' 'A319-100' 'B757-300' 'B787-10'
 'B737-MAX8' 'B737-MAX9' 'B737-700' 'CRJ-550' 'B737-900' 'A321-2NX'
 'A320-200' 'B787-8' 'B757-200' 'B777-2HD' 'ERJ-170' 'B787-9' 'B777-300'
 'B767-400']

Fleet dummy columns created: ['fleet_type', 'fleet_A319-100', 'fleet_A320-200', 'fleet_A321-2NX', 'fleet_B737-700', 'fleet_B737-800', 'fleet_B737-900', 'fleet_B737-MAX8', 'fleet_B737-MAX9', 'fleet_B757-200', 'fleet_B757-300', 'fleet_B767-300', 'fleet_B767-400', 'fleet_B777-2HD', 'fleet_B777-300', 'fleet_B787-10', 'fleet_B787-8', 'fleet_B787-9', 'fleet_CRJ-200', 'fleet_CRJ-550', 'fleet_ERJ-170', 'fleet_ERJ-175']


In [45]:
# Convert datetime columns to proper datetime format
flight_df['scheduled_departure_datetime_local'] = pd.to_datetime(flight_df['scheduled_departure_datetime_local'])
flight_df['scheduled_arrival_datetime_local'] = pd.to_datetime(flight_df['scheduled_arrival_datetime_local'])
flight_df['actual_departure_datetime_local'] = pd.to_datetime(flight_df['actual_departure_datetime_local'])

# Ground time constraint
flight_df['ground_tight'] = (flight_df['scheduled_ground_time_minutes'] - flight_df['minimum_turn_minutes'] <= 5).astype(int)

# Bag ratio (fixed)
bag_grouped = bag_df.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local'])['bag_type'].value_counts().unstack(fill_value=0)
bag_grouped['transfer'] = bag_grouped.get('Transfer', pd.Series(0, index=bag_grouped.index))
bag_grouped['checked'] = bag_grouped.get('Checked', pd.Series(0, index=bag_grouped.index))
bag_grouped['bag_ratio'] = bag_grouped['transfer'] / bag_grouped['checked'].replace(0, np.nan)
bag_grouped = bag_grouped.reset_index()[['company_id', 'flight_number', 'scheduled_departure_date_local', 'bag_ratio']]

# Passenger load
pnr_grouped = pnr_df.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local'])['total_pax'].sum().reset_index(name='total_passengers')

# SSR
remark_with_dates = remark_df.merge(
    flight_df[['company_id', 'flight_number', 'scheduled_departure_date_local']],
    on='flight_number',
    how='left'
)
remark_grouped = remark_with_dates[remark_with_dates['special_service_request'] != 'None'].groupby(['flight_number', 'scheduled_departure_date_local']).size().reset_index(name='ssr_count')

# Merge all dataframes
full_df = flight_df.merge(pnr_grouped, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')
full_df = full_df.merge(bag_grouped, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')
full_df = full_df.merge(remark_grouped, on=['flight_number', 'scheduled_departure_date_local'], how='left')

# Fill missing values
full_df['total_passengers'] = full_df['total_passengers'].fillna(0)
full_df['bag_ratio'] = full_df['bag_ratio'].fillna(0)
full_df['ssr_count'] = full_df['ssr_count'].fillna(0)

# Calculate metrics
full_df['load_factor'] = full_df['total_passengers'] / full_df['total_seats']
full_df['ssr_per_pax'] = full_df['ssr_count'] / full_df['total_passengers'].replace(0, np.nan).fillna(0)

# Departure hour weight
full_df['dep_hour'] = full_df['scheduled_departure_datetime_local'].dt.hour
full_df['dep_weight'] = ((full_df['dep_hour'].between(6, 9) | full_df['dep_hour'].between(17, 20)).astype(int)) * 1.5

# Haul hours
full_df['haul_hours'] = (full_df['scheduled_arrival_datetime_local'] - full_df['scheduled_departure_datetime_local']).dt.total_seconds() / 3600

# Fleet type dummies
fleet_dummies = pd.get_dummies(full_df['fleet_type'], prefix='fleet')
full_df = pd.concat([full_df, fleet_dummies], axis=1)

# Calculate delay minutes
full_df['delay_minutes'] = ((full_df['actual_departure_datetime_local'] - full_df['scheduled_departure_datetime_local']).dt.total_seconds() / 60).fillna(0)

# Build OLS Model - dynamically get fleet columns
fleet_cols = [col for col in full_df.columns if col.startswith('fleet_') and col != 'fleet_type']
feature_cols = ['ground_tight', 'bag_ratio', 'load_factor', 'ssr_per_pax', 'dep_weight', 'haul_hours'] + fleet_cols

# Check data types before modeling
print("Data types of features:")
print(full_df[feature_cols].dtypes)
print("\nAny non-numeric values:")
print(full_df[feature_cols].select_dtypes(include=['object']).columns.tolist())

# Convert all features to numeric, forcing errors to NaN
X = full_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)
y = full_df['delay_minutes']

# Remove any rows where y is NaN or inf
valid_idx = np.isfinite(y)
X = X[valid_idx]
y = y[valid_idx]

model = sm.OLS(y, X).fit()
full_df.loc[valid_idx, 'difficulty_score'] = np.abs(model.predict(X))

print(model.summary())

Data types of features:
ground_tight         int64
bag_ratio          float64
load_factor        float64
ssr_per_pax        float64
dep_weight         float64
haul_hours         float64
fleet_A319-100        bool
fleet_A320-200        bool
fleet_A321-2NX        bool
fleet_B737-700        bool
fleet_B737-800        bool
fleet_B737-900        bool
fleet_B737-MAX8       bool
fleet_B737-MAX9       bool
fleet_B757-200        bool
fleet_B757-300        bool
fleet_B767-300        bool
fleet_B767-400        bool
fleet_B777-2HD        bool
fleet_B777-300        bool
fleet_B787-10         bool
fleet_B787-8          bool
fleet_B787-9          bool
fleet_CRJ-200         bool
fleet_CRJ-550         bool
fleet_ERJ-170         bool
fleet_ERJ-175         bool
dtype: object

Any non-numeric values:
[]


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [46]:
full_df['date'] = pd.to_datetime(full_df['scheduled_departure_date_local'])
full_df['rank'] = full_df.groupby('date')['difficulty_score'].rank(ascending=False)
full_df['percentile'] = full_df.groupby('date')['difficulty_score'].rank(pct=True)
full_df['category'] = pd.cut(full_df['percentile'], bins=[0, 0.33, 0.67, 1], labels=['Difficult', 'Medium', 'Easy'], include_lowest=True)
output_df = full_df[['date', 'flight_number', 'scheduled_arrival_station_code', 'ground_tight', 'bag_ratio', 'load_factor', 'ssr_per_pax', 'dep_weight', 'haul_hours', 'fleet_type', 'difficulty_score', 'rank', 'category']]
output_df.to_csv('test_johndoe.csv', index=False)
logger.info("Difficulty scores saved to test_johndoe.csv")

KeyError: 'Column not found: difficulty_score'

In [47]:
sample_day = full_df[full_df['date'] == full_df['date'].iloc[0]].head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x='flight_number', y='difficulty_score', hue='category', data=sample_day)
plt.title('Top 10 Flights by Difficulty Score - Sample Day')
plt.xticks(rotation=45)
plt.ylabel('Difficulty Score')
plt.show()

ValueError: Could not interpret value `difficulty_score` for `y`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>